In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [ ]:
! pip install datasets transformers sacrebleu
! pip install datasets transformers rouge-score nltk


     |████████████████████████████████| 266kB 8.4MB/s 
     |████████████████████████████████| 2.5MB 41.7MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 122kB 59.7MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 245kB 58.8MB/s 
     |████████████████████████████████| 3.3MB 34.8MB/s 
     |████████████████████████████████| 901kB 55.0MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.14 which is incompatible.


In [ ]:
from datasets import load_dataset
df = load_dataset("cnn_dailymail",'3.0.0',split= 'train[:20%]')
df.to_csv('test.csv')
from datasets import load_dataset, load_metric

raw_datasets = load_dataset('csv', data_files= '/content/test.csv')
metric = load_metric("rouge")

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


Using custom data configuration default-e69e74767ab8a781


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e69e74767ab8a781/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [ ]:
model_checkpoint = "facebook/bart-base"

!pip install sentencepiece

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


     |████████████████████████████████| 1.2MB 8.4MB/s 


In [ ]:
if model_checkpoint in ["facebook/bart-base"]:
    prefix = "Summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 256
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "Summary-generation",
    learning_rate=10e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Unnamed: 0, article, highlights, id.
***** Running training *****
  Num examples = 57423
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10767
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Step,Training Loss
500,2.687600
1000,2.544800
1500,2.483000
2000,2.456000
2500,2.443500
3000,2.414900
3500,2.386000
4000,2.166300
4500,2.102300
5000,2.126400


Saving model checkpoint to Summary-generation/checkpoint-500
Configuration saved in Summary-generation/checkpoint-500/config.json
Model weights saved in Summary-generation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Summary-generation/checkpoint-500/tokenizer_config.json
Special tokens file saved in Summary-generation/checkpoint-500/special_tokens_map.json
Saving model checkpoint to Summary-generation/checkpoint-1000
Configuration saved in Summary-generation/checkpoint-1000/config.json
Model weights saved in Summary-generation/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in Summary-generation/checkpoint-1000/tokenizer_config.json
Special tokens file saved in Summary-generation/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to Summary-generation/checkpoint-1500
Configuration saved in Summary-generation/checkpoint-1500/config.json
Model weights saved in Summary-generation/checkpoint-1500/pytorch_model.bin
tokenizer config file saved i

TrainOutput(global_step=10767, training_loss=2.152003374274222, metrics={'train_runtime': 3840.2905, 'train_samples_per_second': 44.858, 'train_steps_per_second': 2.804, 'total_flos': 4.611409896141619e+16, 'train_loss': 2.152003374274222, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/drive/My Drive/Code/summary")
tokenizer.save_pretrained("/content/drive/My Drive/Code/summary")

Configuration saved in /content/drive/My Drive/Code/summary/config.json
Model weights saved in /content/drive/My Drive/Code/summary/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Code/summary/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Code/summary/special_tokens_map.json


('/content/drive/My Drive/Code/summary/tokenizer_config.json',
 '/content/drive/My Drive/Code/summary/special_tokens_map.json',
 '/content/drive/My Drive/Code/summary/vocab.json',
 '/content/drive/My Drive/Code/summary/merges.txt',
 '/content/drive/My Drive/Code/summary/added_tokens.json',
 '/content/drive/My Drive/Code/summary/tokenizer.json')